<a href="https://colab.research.google.com/github/BarryLiu-97/Pytorch-Tutorial/blob/master/09_Advanced_CNN_GoogleNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 编写GoogleNet

In [ ]:
import torch
from torch import nn
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.optim as optim
import time

In [ ]:
batch_size = 64
transform = transforms.Compose([
  transforms.ToTensor(),              #将数据转换为channel×width×height格式，为了更高效地进行运算
  transforms.Normalize((0.1307, ), (0.3081, ))  #均值和标准差，用于数据标准化，这是对MNIST进行计算后得到的结果，已经算好了
])

train_dataset = datasets.MNIST(root='../dataset/mnist',
                train=True, download=True,
                transform = transform)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=batch_size)

test_dataset = datasets.MNIST(root='../dataset/mnist',
                train=False, download=True,
                transform = transform)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=batch_size)

Extracting ../dataset/mnist/MNIST/raw/train-images-idx3-ubyte.gz to ../dataset/mnist/MNIST/raw


Extracting ../dataset/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ../dataset/mnist/MNIST/raw


Extracting ../dataset/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ../dataset/mnist/MNIST/raw


Extracting ../dataset/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../dataset/mnist/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [ ]:
class InceptionA(nn.Module):
  def __init__(self, in_channels):
    super(InceptionA, self).__init__()
    self.branch_pool = nn.Conv2d(in_channels, 24, kernel_size=1)

    self.branch1x1 = nn.Conv2d(in_channels, 16, kernel_size=1)

    self.branch5x5_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
    self.branch5x5_2 = nn.Conv2d(16, 24, kernel_size=5, padding=2)

    self.branch3x3_1 = nn.Conv2d(in_channels, 16, kernel_size=1)
    self.branch3x3_2 = nn.Conv2d(16, 24, kernel_size=3, padding=1)
    self.branch3x3_3 = nn.Conv2d(24, 24, kernel_size=3, padding=1)
  
  def forward(self, x):
    branch_pool = F.avg_pool2d(x, kernel_size=3, stride=1, padding=1)
    branch_pool = self.branch_pool(branch_pool)

    branch1x1 = self.branch1x1(x)

    branch5x5 = self.branch5x5_1(x)
    branch5x5 = self.branch5x5_2(branch5x5)

    branch3x3 = self.branch3x3_1(x)
    branch3x3 = self.branch3x3_2(branch3x3)
    branch3x3 = self.branch3x3_3(branch3x3)

    outputs = [branch1x1, branch5x5, branch3x3, branch_pool]
    # 因为输出的张量维度是B x C x W x H，我们要按照通道进行拼接(也意味着各个输出的W和H要一样)
    return torch.cat(outputs, dim=1)

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
    self.conv2 = nn.Conv2d(88, 20, kernel_size=5) # 88 = 24*3 + 16

    self.incep1 = InceptionA(in_channels=10)
    self.incep2 = InceptionA(in_channels=20)

    self.mp = nn.MaxPool2d(2)
    self.fc = nn.Linear(1408, 10)

  def forward(self, x):
    in_size = x.size(0)
    x = F.relu(self.mp(self.conv1(x)))
    x = self.incep1(x)
    x = F.relu(self.mp(self.conv2(x)))
    x = self.incep2(x)
    x = x.view(in_size, -1)
    x = self.fc(x)
    return x

In [ ]:
model = Net()
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)  #冲量值设置为0.5，优化训练过程

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # 若当前cuda是可行的，使用第一个可见的设备(即GPU)
model.to(device) # 参数、缓存等，所有的模块放入cuda，使用GPU

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(88, 20, kernel_size=(5, 5), stride=(1, 1))
  (incep1): InceptionA(
    (branch_pool): Conv2d(10, 24, kernel_size=(1, 1), stride=(1, 1))
    (branch1x1): Conv2d(10, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch5x5_1): Conv2d(10, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch5x5_2): Conv2d(16, 24, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (branch3x3_1): Conv2d(10, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch3x3_2): Conv2d(16, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (branch3x3_3): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (incep2): InceptionA(
    (branch_pool): Conv2d(20, 24, kernel_size=(1, 1), stride=(1, 1))
    (branch1x1): Conv2d(20, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch5x5_1): Conv2d(20, 16, kernel_size=(1, 1), stride=(1, 1))
    (branch5x5_2): Conv2d(16, 24, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
   

In [ ]:
def train(epoch):
  running_loss = 0.
  for batch_idx, data in enumerate(train_loader, 0):
    inputs, target = data
    inputs, target = inputs.to(device), target.to(device)  # 转移到cuda，且在同一块显卡上
    optimizer.zero_grad()

    # forward + backard + update
    outputs = model(inputs)
    loss = criterion(outputs, target)
    loss.backward()
    optimizer.step()

    running_loss += loss.item()
    if batch_idx % 300 == 299:
      print('[%d, %5d] loss: %.3f' % (epoch+1, batch_idx + 1, running_loss / 2000))
      running_loss = 0.0

In [ ]:
def test():
  correct = 0
  total = 0
  with torch.no_grad():
    for data in test_loader:
      inputs, target = data
      inputs, target = inputs.to(device), target.to(device)
      outputs = model(inputs)
      _, predicted = torch.max(outputs.data, dim=1)
      total += target.size(0)
      correct += (predicted == target).sum().item()
  print('Accuracy on test set: %d %% [%d/%d]' % (100*correct / total, correct, total))

In [ ]:
start = time.time()
for epoch in range(10):
  train(epoch)
  test()
end = time.time()
print(str(end-start) + 's')

[1,   300] loss: 0.130
[1,   600] loss: 0.027
[1,   900] loss: 0.019
Accuracy on test set: 96 % [9686/10000]
[2,   300] loss: 0.016
[2,   600] loss: 0.014
[2,   900] loss: 0.013
Accuracy on test set: 97 % [9770/10000]
[3,   300] loss: 0.011
[3,   600] loss: 0.011
[3,   900] loss: 0.011
Accuracy on test set: 98 % [9809/10000]
[4,   300] loss: 0.009
[4,   600] loss: 0.009
[4,   900] loss: 0.010
Accuracy on test set: 98 % [9810/10000]
[5,   300] loss: 0.008
[5,   600] loss: 0.009
[5,   900] loss: 0.008
Accuracy on test set: 98 % [9846/10000]
[6,   300] loss: 0.007
[6,   600] loss: 0.008
[6,   900] loss: 0.008
Accuracy on test set: 98 % [9855/10000]
[7,   300] loss: 0.006
[7,   600] loss: 0.007
[7,   900] loss: 0.007
Accuracy on test set: 98 % [9852/10000]
[8,   300] loss: 0.006
[8,   600] loss: 0.007
[8,   900] loss: 0.006
Accuracy on test set: 98 % [9872/10000]
[9,   300] loss: 0.006
[9,   600] loss: 0.006
[9,   900] loss: 0.006
Accuracy on test set: 98 % [9868/10000]
[10,   300] loss: 0